# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [13]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 2

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    
    time.sleep(SCROLL_PAUSE_TIME)
    
    new_height = browser.execute_script('return document.body.scrollHeight')
    
    print(f'last_height : {last_height}')
    print(f'new_height : {new_height}')
    
    if new_height == last_height:
        print(f'到達頁面底端')
        break
    print(f'網頁更新中...')
    
    last_height = new_height
    '''
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''

last_height : None
new_height : 10853
網頁更新中...
last_height : 10853
new_height : 13185
網頁更新中...
last_height : 13185
new_height : 15547
網頁更新中...
last_height : 15547
new_height : 17909
網頁更新中...
last_height : 17909
new_height : 20160
網頁更新中...
last_height : 20160
new_height : 22434
網頁更新中...
last_height : 22434
new_height : 23772
網頁更新中...
last_height : 23772
new_height : 23822
網頁更新中...
last_height : 23822
new_height : 23822
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [14]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [16]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 口罩再掀搶購潮　王美花：通路加緊鋪貨、徵用維持800萬片不變

▲經濟部長王美花表示，口罩實名制徵用數量仍維持800萬片，會一直持續到今年底。（圖／記者林敬旻攝）
記者林淑慧／台北報導
國內口罩搶購潮再起，經濟部長王美花今（12）日表示，口罩實名制徵用數量仍維持800萬片，上周部分藥局通路很快就賣光，指揮中心已針對通路端加緊鋪貨，企業自行販售者為自由商業行為，經濟部尊重市場機制。
另外，市場上傳出有中國大陸製口罩仿冒MIT口罩高價販售，王美花於受訪時表示，官方針對仿冒MIT的口罩從進口端就有嚴格審查，包括財政部海關也在把關，若有違法將面對刑責。
經濟部官員也提醒，口罩實名制預購會一直持續到年底，一樣是14天可買9片，每片5元，第14期的口罩實名制預購預計下周一超商開放領取，呼籲全民多加利用。
經濟部官員表示，口罩實名制的預購很方便，只要記得在期限內完成繳費，領取時間一到，就可以帶著健保卡到指定通路領取台灣製造的口罩，而且口罩實名制會一直持續到年底，請大家不用擔心。
另外，經濟部估算，口罩業者每周外銷量約為2000萬片，官員指出，這樣的量其實不算多，而實際上　國內口罩需求仍維持高檔，許多外銷訂單也因此被國內需求量排擠。
 
----------------------------------------------------------------------

[1] 保德信投信在台邁第28年　持續接軌國際、深耕台灣

▲保德信投信在台灣邁入第28年。（圖／記者湯興漢攝）
記者陳心怡／台北報導
----------------------------------------------------------------------

[2] 實名制口罩　經濟部長王美花：本週民眾應該可以買得到

▲經濟部長王美花。（圖／記者湯興漢攝）
記者吳靜君／台北報導
經濟部長王美花今（12）日表示，由於上週因為有幾個疑義的案例（新冠肺炎案例）所以導致民眾到藥局實名制買口罩有買步到的情況；但是本週應該鋪貨補足。
王美花表示，目前政府仍然維持每日徵用800萬片；至於近期市面上、海關都有發現中國大陸製的口罩有壓製「MIT」的情況，

新聞節目中心／綜合報導
SOGO經營權纏鬥18年，意外爆發台灣史上最大國會貪瀆的案外案！《公司法》的SOGO條款究竟在吵什麼？立委在修法過程中又採到哪些紅線以致觸法？由資深司法記者蘇位榮主持的《行動法庭》，昨（11）日邀請台北律師公會人權委員會委員黃帝穎律師、太平洋SOGO第一代查帳會計師巫鑫，解析涉賄立委的法律爭議和《公司法》的SOGO條款的前世今生。
▲ 太平洋SOGO第一代查帳會計師巫鑫。（圖／行動法庭提供）
太平洋SOGO第一代查帳會計師巫鑫指出，太流公司起初資本額僅1000萬，其中600萬是由太平洋建設董事長章民強信託登記給李恆隆。民國90年納莉風災，太平洋建設傳出經營困難，李恆隆竟「引清兵入關」，找遠東集團徐旭東認購太流公司40億的增資案，以提供資金紓困。
▲ 會計師巫鑫講解，太平洋SOGO經營權爭奪戰。（圖／巫鑫提供提供）
2015年，《公司法》第9條修法（俗稱SOGO條款），補上法律漏洞，但因新版條文無溯及既往的規定，李恆隆無法依法撤銷太流公司的資本額登記，因此他積極透過立委推動《公司法》第9條增訂「溯及既往」的規定，在過程中因行賄立委，釀成台灣史上最大的立委集體收賄案。
▲ 《公司法》第9條成為爭奪太平洋SOGO百貨經營權的戰場。（圖／行動法庭提供，以下亦同）
立委踩紅線！黃帝穎：對台灣民主造成非常大的傷害！
對於立委爆發集體收賄的弊案，律師黃帝穎沈痛表示，「對台灣的民主政治而言，我認為是非常大的傷害！」因為本案會將打擊人民對國會的信任，當人們往回查閱過去那些立委在修法時的慷慨激昂、正義凜然，人們會懷疑立委的發言是否有其他動機或利益輸送？
黃帝穎指出，李恆隆如果只是一般的遊說，不會有違法的問題，但李恆隆被檢調發現他在遊說過程中有「利益輸送」。
「只要是收了錢就不法。」黃帝穎強調，「其實修法本來就是立委職權，但當立委跟李恆隆和白手套郭克銘收錢，就會被檢調認為有貪污的嫌疑。」
至於如果有立委只拿錢不辦事，仍觸犯《貪污治罪條例》利用職務詐取財物罪，還是違法。
檢調發動監聽、搜索、羈押　須先說服法官
目前立委集體收賄案，蘇震清、廖國棟、陳超明、趙正宇等四位立委遭羈押禁見，徐永明則交保獲釋。
黃帝穎表示，立委被收押，至少需要經過三關。檢調單位若要發動監聽、搜索、羈押，在每一個環節，都必須拿出足以說服法官的證據，法官才會發出監聽票、搜索票和允許羈押。
對於本次

▲台新金控8月11日晚間召開重大訊息說明會。（圖／記者湯興漢攝）
記者紀佳妘／台北報導
保德信人壽規劃出售台灣子公司，並於6月1日截止投標，台新金控今（11）日晚間宣布，董事會決議將以55億元為基準取得保德信人壽100%股權，正式跨足保險業務領域，完備金控第三具業務動力引擎，後續本公司於備妥相關文件後將向主管機關正式提出申請核准本項股權交易。
台新金控總經理林維俊今晚在證交所重大訊息說明會上表示，擬取得保德信人壽，出價金額為55億元，但考量市場利率環境，將引用價格調整機制，最高為30億元，此案待主管機關核准同意，為達成長期經營目標，員工將全數留任，且保戶權益也不受影響，將維持不變。
▲台新金控總經理林維俊宣布將正式併購保德信人壽。（圖／記者湯興漢攝）
台新金控集團之前雖無保險子公司，但長期經營銀行保險（Bancassurance）業務成效顯著，將來納入保德信人壽後，仍將秉持長期經營的初心，規劃經營保險業務；對於長期經營策略所需的資金，台新金控指出，會有妥善的短期及長期規劃來滿足主管機關的要求。
在台耕耘30年的保德信人壽規劃出售台灣子公司，並於6月1日截止投標，市場傳出有意併購壽險業的金融機構，包括台新金、第一金、聯邦銀行等可能都是潛在買家，最後由台新金搶親成功。
此外，金管會過去審查壽險公司的買家有5大原則，首先是承諾保障保戶及員工權益，其次為資金來源須符合法令規定及財務健全性，第三為須有專業能力經營保險業；第四是長期經營承諾；第五，必須有財務能力因應未來增資需求。換句話說，未來不論花落誰家，得標的買方也須經金管會審查通過才算數。
----------------------------------------------------------------------

[26] Fossil Gen 5 下周三有重大更新！　睡眠追蹤、血氧偵測一次擁有

記者謝仁傑／綜合報導
Fossil 第5代智慧手錶「Gen 5」的用戶，將在8月19日透過軟體更新，一次提供許多新功能，包含睡眠追踪、有氧運動偵測和VO2 max監控（最大攝氧量、血氧偵測）等功能。
外媒報導，Fossil 手錶會有全新的應用程式「Wellness」，用戶必須在手機下載，或透過手錶使用，還可以在運動的過程中追踪VO2的最大估計值，如此一來用戶就能輕鬆接收到所有身體健康相關的訊息。
儘管 Foss

記者謝仁傑／綜合報導
蘋果先前就已經為 iOS、iPadOS、macOS 和 tvOS 的下一版重大更新發布了beta版本，現再將 watchOS 增加到公開Beta版中。
watchOS 7 版本包含新的錶面，且可以與朋友共享錶面，或從網路下載自設錶面佈局的簡易方法。然而今年最值得關注的一項功能就是「睡眠追踪」，watchOS 7 新增「睡眠」應用程式，每晚它將向用戶顯示有多少時間處於睡眠狀態，儘管蘋果尚未深入研究睡眠質量和不同的睡眠周期。
Apple Watch 透過 watchOS 7 獲得更多健康相關的追踪訊息，包括舞蹈、功能強度訓練、核心訓練和冷卻活動的指標；另外，Google Maps 也重新支援 Apple Watch，且新增自行車路線。
Apple Watch 會自動偵測用戶洗手，並且還提供20秒的洗手計時，此外還有其他較小但有用的改進，例如設備上的聽寫功能，使語音轉換文字速度更快，以及 Siri 的翻譯支援。
蘋果官網上有 watchOS 新功能的完整列表，用戶只要安裝 watchOS 7 的Beta版，就可以自己進行測試。‌watchOS 7 ‌僅限於‌ Apple Watch‌ Series 3、Series 4 和 Series 5 型號，與‌ Series 1 和 Series 2 並不兼容。
----------------------------------------------------------------------

[38] 證交所：杏輝明日起恢復交易

▲證交所公告杏輝明日起將恢復交易。（圖／《ETtoday》資料照）
記者陳心怡／台北報導
證交所今日公告，杏輝（1734）已於8月11日公布暫停交易事由之相關重大訊息，其上市普通股自明（12）日起恢復交易。
杏輝旗下杏國（4192）今日召開重大訊息記者會，公布研發中新藥SB05PC全球胰臟癌三期人體臨床試驗期中分析（IA）結果，指出獨立資料監察委員會(Data Monitoring Committee, DMC) 未提及安全性問題，並建議無需修改計畫書，依原計畫繼續執行臨床試驗。
----------------------------------------------------------------------

[39] 感測器商調查耳機市場　近三成消費者需要

▲台指期8月11日走勢。（圖／取自期交所）
記者余弦妙／台北報導
台指期今（11）日開盤下跌29點，來到12825點，早盤開低後，在接近午盤一度反攻，甚至翻紅來到12874點，不過上方壓力仍大，反攻格局維持不久，又再度拉回，接著一路下跌，終場以12737點作收，下跌117點，與現貨相較，逆價差43.19點。
展望後市，統一期貨投顧部經理廖恩平分析，最近這段時間台股還是在相對高檔的位置做整理，但這邊也有注意一點，若是成交量能出現遞減的話，上檔的反壓就會比較重一些，今天這幾天已經出現連續三天量能下跌，同樣的也伴隨著加權指數有稍微的壓回，當然目前看來還不緊迫，但這邊仍要提醒大家，若是量能遲遲無法放量，也就是無法超過5日均量的話，上方的壓力就會越來越重。
廖恩平更提到，另一個觀察重點是，加權指數一直到上周已經出現連8紅，這樣的連8紅，自1987年以來僅出現過8次，其中僅有一次在結束連續周線收紅，還能夠維持往上小幅上漲一點，其餘的7次都是在結束連8紅後，基本上都是壓回，甚至其中還有5次在結束連續周線收紅之後的12周下跌幅度都超過10%。
廖恩平也說，雖然目前看起來還算是強勢，但目前還是要去提防說一旦周線連紅走勢結束之後，它會有一些修正，畢竟目前看來是漲多，難免接下來會出現賣壓。
----------------------------------------------------------------------

[51] 大同市場派向經濟部喊話　王美花：股臨會審議本周作出決議

▲大同公司經營權之爭持續延燒。（圖／記者湯興漢攝）
記者林淑慧／台北報導
大同公司經營權之爭持續延燒，市場派之一、三圓建設董事長王光祥召開記者會向經濟部喊話，經濟部長王美花今（11）日受訪時表示，內部已加緊審議進度，重申本周將針對市場派申請股臨會作出決議，並一併對外說明。
大同公司市場派、三圓建設董事長王光祥昨天召開記者會表示，若順利入主大同，將推舉林文淵擔任董事長，要仰賴林文淵帶領大同專業團隊，為公司帶來獲利，並向經濟部喊話「勇於任事、不要再推拖。」
經濟部長王美花回應表示，大同案社會各界矚目，市場派兩造都很關心申請股臨會的進度，經濟部同仁已在加緊作業中，強調會根據相關法律意見做分析判斷，「這個禮拜一定會盡快做出來」。
大同市場派林宏信等人在7月9日依據公司法第173條第4項，向經濟部送件申

----------------------------------------------------------------------

[63] 台籍男子狀告立訊董座王來春！聲稱價值840億股票被吞　遭立訊否認

▲立訊精密傳出重大股權糾紛。（圖／取自立訊精密官網）
記者王曉敏／綜合報導
立訊精密傳出重大股權糾紛！一位自稱原任職立訊精密集團管理中心台籍高管吳政衛，日前向陸媒投訴，聲稱與立訊董事長王來春間有股權代持糾紛，涉及立訊精密5.3％股權，市值達人民幣199.93億元（約新台幣840億元）。不過立訊昨（10）日表示，王來春與吳政衛不存在任何關聯關係，也沒有股份代持情況。
綜合陸媒報導，吳政衛日前向《中國經濟網》投訴，並出示與王來春於2011年3月23日簽署的股權承諾聲明書，指稱他當年透過王來春代為持有香港立訊有限公司8%的股權。據吳政衛說法，2007年，王來春力邀他以經營者身分加入立訊，以協助立訊精密公司上市，並承諾年薪及立訊精密公司股份。但自立訊精密股票於2010年上市後，吳政衛曾多次試圖與王來春商量代持股權變現事宜均未果。
2019年6月13日，吳政衛向深圳中院控告王來春、立訊精密以及立訊有限公司；2020年6月，深圳中院已向立訊精密送達了本案相關材料，但上市公司迄今尚未公告。
立訊於6日向《中國經濟網》表示，公司領導正在處理此事，還沒有具體情況可以告知。不過據《中證網》報導，立訊精密證券部昨日回應表示，控股股東王來春與吳政衛不存在任何關聯關係，也不存在股份代持情況。
且據《中國經濟網》報導，吳政衛稱自己加入立訊後，即作為立訊精密公司所有投資公司的最高級別顧問，負責公司的財稅規劃、協助公司上市等工作。但立訊精密回應指出，公司上市以來，一直沒有名字為吳政衛的高管存在，並稱「經王來春確認，她沒有簽過這個股份代持承諾書。」
而2010年，立訊精密上市招股書上的公司控股股東，是由王來春、王來勝兄妹各持有立訊有限公司50%的股權，是立訊精密的實際控制人。
----------------------------------------------------------------------

[64] 俄羅斯反托拉斯調查　認定蘋果濫用市場地位

▲蘋果在俄羅斯反托拉斯調查中被判有罪。圖為蘋果總部。（圖／達志影像／美聯社）
記者林鼎閎／綜合報導
蘋果已經看到

▲振興三倍券刺激民生消費熱潮，統一超七月營收達230.6億元，創史上單月新高。（圖／記者張一中攝）
記者林淑慧／台北報導
振興三倍券刺激民生消費熱潮，超商雙雄統一超、全家七月營收分別達230.6億元、78.3億元，雙雙創下史上單月新高，主因在於超商本業穩定成長，營收報喜也帶動台股今（11）日開盤股價上攻，吸引買盤持續敲進。
統一超109年7月合併營業額230.6億元，較去年同期成長3.02%。7-ELEVEN受惠持續推出創新店型，「7-ELEVEN DAY」、振興三倍券優惠等活動擴大買氣，並掌握鮮食、飲料、出版品等銷售商機，超商本業穩定成長；轉投資企業包括統一生活、博客來、統一速達、統一百華及酷聖石冰淇淋等表現亮眼，整體營運穩健向上。
其中， 7-ELEVEN扮演社會廚房角色，致力打造安心美味的鮮食，更將即食配菜、小吃等商品以吊掛式陳列，其中生鮮食品為主要成長動能，成長近六成，以主打健康概念的雞胸肉、溫泉蛋、玉米棒等最受歡迎；飲料部分，因國旅與夏季高溫，水、茶及運動飲料等需求提高，加上飲料抽抽樂與振興券兩倍用等獨家活動帶動銷售成長近一成。
▲全家便利商店推出中元節優惠方案搶市。（圖／全家提供）
全家便利商店109年度7月合併營收為78.3億元，年增13.52%，主因在於店數較去年同期增加7%，鮮食成長一成五，皆有效挹注營收。三倍券與多重優惠活動，各式話題新品連番上市，帶動鮮食中的現煮飲料品群，在七月的多重利多因素下交出成長近三成的成績。
 
----------------------------------------------------------------------

[77] 新台幣以平盤開出後　強彈1.35角至29.385元

▲新台幣開盤以平盤29.52元開出。（圖／記者湯興漢攝）
記者陳心怡／台北報導
台北外匯市場新台幣兌美元匯率今（11）日開盤以平盤29.52元開出，不過，在熱錢不退下，新台幣由貶轉升，早盤來到29.385元，升值1.35角。
----------------------------------------------------------------------

[78] 大立光啟動79除息行情 早盤填息上漲逾0.5%！

▲大立光執行長林恩平。（圖／記者范綱儀攝）
記者陳心怡／台北報導
股王大立光（3008）今（11）

▲EPFR統計顯示，上周主要債市皆迎來資金淨流入。（圖／實習記者方敬攝）
財經中心／台北報導
肺炎疫苗研發持續有所進展、美國就業數據樂觀，過去一周市場情緒正面，風險性債券多有正面表現。根據美銀美林引述EPFR統計顯示，上周主要債市皆迎來資金淨流入，包括投資級債、新興市場債及高收益債皆較前一周有明顯增幅，其中，高收益債淨流入金額更較前周成長近5.7倍，來到44.1億美元，動能強勁。
安聯投信指出，美國藥廠研發的新冠肺炎疫苗臨床初步實驗數據正向，市場樂觀預期疫苗有望在今年底至明年初間問世，加上美國最新出領失業救濟金人數及7月非農就業數據優於預期，激勵信用債上周普遍走揚。觀察各式債券資產，全球高收益債單周上漲約0.58%，避險性債券如全球政府債券則是走跌約0.22%，反應市場情緒。
安聯美國短年期高收益債券基金經理人謝佳伶表示，近來雖有二次疫情及美中關係緊張等不確定性壟罩，但全球市場風險情緒仍偏正向，主要市場股市包括美國、中國及歐洲等股市近來皆有不錯表現，然而對許多偏好穩定收益、風險取向較為謹慎的投資人來說，債券資產仍是相較股市為佳的投資選擇。
謝佳伶表示，綜觀各式債券資產，政府公債雖過去長期有較為穩定的避險形象，但在市場變化快速，灰犀牛潛伏的此時，公債價格近來也有不小起伏；舉例來說，公債中極具代表性的美國10年期公債自今年初至今，殖利率便在0.5至1.0%間起伏，進而使得公債及與其高度相關的資產波動幅度不小。
在信用債市方面，謝佳伶指出，投資級債雖有相對較低波動及較違約率的特色，但年初至今以淨流入將近600億美元，大幅高於高收益債及新興市場債，在資金大量湧入的情形下，殖利率也持續被壓低，難以提供誘人的收益率。
相較之下，高收益債因有仍優於投資級債與公債的殖利率，加上風險取向雖略高於投資級債，但仍較股票資產為低，相較之下為在風險屬性與穩定收益特性之中，位處相對中間及平衡的一個選擇。
然而在高收益債市中，因仍有市場、產業等變數，不同券種表現也有不小差距。故在布局高收益債時，謝佳伶建議首先應關注在全球中綜合國力相對強勢、且市場成熟度高的美國相關資產；在產業方面，則可鎖定位處低景氣循環產業、較不受國際貿易變化影響的內需及支援類型產業如保全、電信通訊等，降低貿易摩擦衝擊。
最後，謝佳伶表示，也可挑選存續期較短（約在2年上下）的券種，如此便有望受惠於較快到期的優勢，降低持有期間內資

In [17]:
browser.close()